In [1]:
from dendron import *
from dendron.hf.actions.causal_lm_action import CausalLMActionConfig, CausalLMAction

In [2]:
cfg = CausalLMActionConfig(load_in_8bit=True, 
                           max_new_tokens=64, 
                           do_sample=True, 
                           top_p=0.95, 
                           model_name = 'v1olet/v1olet_merged_dpo_7B')
                           #model_name="mistralai/Mistral-7B-Instruct-v0.2")
node = CausalLMAction("lm_action", cfg)
tree = BehaviorTree(node)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#prompt = "<s>[INST]{query}[/INST]" For Mistral

prompt = """{system}
### Instruction:
{query}

### Response:
"""

In [36]:
import re
def trunc(node, s):
    # for mistral:
    #m = re.search(r'\[INST\](.*)\[/INST\](.*)', s, re.DOTALL)
    #return m.group(2)

    m = re.search(r'### Response:(.*)', s, re.DOTALL)
    return m.group(1).strip()

tree.root.set_post_tick(trunc)

In [119]:
tree.blackboard_set("in", prompt.format(system="You are busy and don't have much time to be verbose.", query="The following is a headline from a newspaper: 'History-making SpaceX booster mostly destroyed in post-flight topple.' Is this about a spacecraft? Give a one-word yes or no."))

In [120]:
status = tree.tick_once()
if status == NodeStatus.SUCCESS:
     print(tree.blackboard["out"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Yes


In [118]:
print(tree.blackboard)

Key              | Description      | Type             | Value            |
in               | Autogenerated en | <class 'str'>    | You are busy and | 
out              | Autogenerated en | <class 'str'>    | No               | 



In [115]:
tree.blackboard['out']

'Yes.'